## Generating Synthetic Data (CTGAN)

## Install CTGAN

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Install CTGAN, generative model for tabular data.
my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)
!pip install --target=$pack_path ctgan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ctgan-0.6.0-py2.py3-none-any.whl (24 kB)
  Using cached rdt-1.2.1-py2.py3-none-any.whl (61 kB)
     |████████████████████████████████| 24.3 MB 82.5 MB/s 
     |████████████████████████████████| 24.8 MB 1.4 MB/s 
     |████████████████████████████████| 15.7 MB 54.5 MB/s 
     |████████████████████████████████| 40 kB 6.7 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:46tcmalloc: large alloc 1147494400 bytes == 0x3d06e000 @  0x7feebab2e615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 890.2 MB 7.0 kB/s 
     |████████████████████████████████| 11.3 MB 48.2 MB/s 
     |█████████████████████████

In [ ]:
# set device: GPU
import torch
torch.cuda.current_device()

0

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import pickle
from collections import defaultdict
from itertools import islice, combinations
from datetime import datetime as dt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt

from ctgan import CTGAN


In [ ]:
# # Control Width of Jupyter Cells
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# t_node_interactivity = "all"
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

# print('ready to run')
# # Log Running Time
# startTime = time.time()

ready to run


## Preprocess Source Data

In [7]:
%cd drive/MyDrive/Customs-Declaration-Datasets-en/

/content/drive/MyDrive/Customs-Declaration-Datasets-en


In [14]:
# Load train data
df_raw=pd.read_csv('./data/df_syn_eng.csv', encoding='utf-8-sig')

In [15]:
# Select columns to use
df_org_merge=df_raw[["Declaration ID", "Date", "Office ID", "Process Type", "Import Type", "Import Use", "Payment Type", 
                  "Mode of Transport", "Declarant ID", "Importer ID", "Seller ID", "Courier ID", 
                  "HS10 Code", "Country of Departure", "Country of Origin",  "Tax Rate", "Tax Type", 
                   "Country of Origin Indicator", "Net Mass", "Item Price", "Fraud", "Critical Fraud"]]

In [16]:
df_org_merge['Date']=df_org_merge['Date'].astype(str)
df_org_merge['Date']=pd.to_datetime(df_org_merge['Date'])
df_org=df_org_merge.sort_values(by=['Date'], axis = 0)

In [19]:
# Sampling 3000 Train Data from Source Data
df_sample=df_org.sample(3000, replace=False)
df_sample.to_csv('./data/df_sample.csv', index=False, encoding='utf-8-sig')

In [20]:
# Aggregate Reletive Columns
cols = ['HS10 Code', 'Country of Departure', 'Country of Origin', 'Tax Rate', 'Tax Type', 'Net Mass','Fraud', 'Critical Fraud']
df_sample['Aggregated'] =df_sample[cols].apply(lambda row: '^'.join(row.values.astype(str)), axis=1)
df_sample=df_sample.drop(cols, axis=1)
df_sample=df_sample.drop(['Item Price'], axis=1)

In [21]:
df_sample['Date']=df_sample['Date'].astype('str')

# Run CTGAN

In [22]:
categorical_columns =['Declaration ID', 'Date','Process Type','Declarant ID','Importer ID','Seller ID',
                      'Courier ID','Country of Origin Indicator','Aggregated']

In [ ]:
# from ctgan import CTGANSynthesizer
# from ctgan import load_demo

In [23]:
ctgan = CTGAN(verbose=True)
ctgan.fit(df_sample, categorical_columns, epochs = 100)

Epoch 1, Loss G:  5.4992,Loss D: -0.1092
Epoch 2, Loss G:  5.6007,Loss D: -0.2241
Epoch 3, Loss G:  5.4361,Loss D: -0.2843
Epoch 4, Loss G:  5.6444,Loss D: -0.2499
Epoch 5, Loss G:  5.7653,Loss D: -0.3303
Epoch 6, Loss G:  5.5681,Loss D: -0.3937
Epoch 7, Loss G:  5.6935,Loss D: -0.3909
Epoch 8, Loss G:  5.8868,Loss D: -0.4679
Epoch 9, Loss G:  5.8095,Loss D: -0.5463
Epoch 10, Loss G:  5.7155,Loss D: -0.4423
Epoch 11, Loss G:  5.5077,Loss D: -0.3274
Epoch 12, Loss G:  5.7927,Loss D: -0.3641
Epoch 13, Loss G:  5.5797,Loss D: -0.1788
Epoch 14, Loss G:  5.2168,Loss D:  0.0635
Epoch 15, Loss G:  5.0230,Loss D:  0.0454
Epoch 16, Loss G:  4.9736,Loss D:  0.0331
Epoch 17, Loss G:  5.2149,Loss D:  0.0145
Epoch 18, Loss G:  4.9043,Loss D: -0.0628
Epoch 19, Loss G:  4.8072,Loss D:  0.0681
Epoch 20, Loss G:  4.4656,Loss D:  0.1701
Epoch 21, Loss G:  4.7738,Loss D:  0.1084
Epoch 22, Loss G:  4.5923,Loss D:  0.0197
Epoch 23, Loss G:  4.6754,Loss D:  0.0669
Epoch 24, Loss G:  4.5870,Loss D:  0.0349
E

In [24]:
# Create data as many as the number of sampled data
count_row = df_sample.shape[0] 
df_syn = ctgan.sample(count_row)

In [25]:
df_syn

,Declaration ID,Date,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,Country of Origin Indicator,Aggregated
0,48649605,2020-10-04,31,B,11,21,19,40,E3Q8ABV,FAQBBWS,3KKQWHO,NaN,G,9031809099^PH^JP^1.3^C^11.9^1^1
1,83990938,2021-05-22,41,B,11,21,44,10,86E3KB0,B6SDM52,I86AB6I,NaN,E,6106201000^VN^VN^0.0^FVN1^79.8^0^0
2,50667114,2020-09-03,40,B,11,21,43,10,X09KN33,IKSA5YW,RB6HURK,NaN,E,2906199000^CN^CN^0.0^FCN1^100.0^0^0
3,11390124,2020-08-12,21,B,84,21,11,10,MRTAD2U,0T9AY7N,7PN0VA5,NaN,G,8301401010^CN^CN^8.0^A^6.0^0^0
4,51191871,2020-09-10,20,B,11,21,11,10,IBG3A46,3X6GY58,842QQZ5,NaN,G,6403999000^IT^IT^0.0^FEU1^2.7^0^0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,92685568,2021-02-01,32,B,10,21,45,11,OZB7KED,Y7K2P0D,NaN,NaN,G,6403999000^MT^CN^13.0^A^100.0^1^1
2996,24146768,2020-10-06,40,B,85,12,44,10,AZI5L7O,89P0ZBF,6WOOZSN,PIAN6F,Y,6206401000^CN^CN^13.0^A^217.0^0^0
2997,98823569,2021-02-01,41,B,10,21,45,41,ACL5K0W,51RX2UZ,NME7L1Q,NaN,G,6204599000^ES^CN^13.0^A^13.2^0^0
2998,41805945,2020-07-30,40,B,11,21,11,11,H1ZUISK,5W9THF3,U11PIYH,NaN,E,6104520000^FR^TR^13.0^A^10.0^0^0


In [26]:
# Split aggregated column to original form
df_syn['HS10 Code']=df_syn["Aggregated"].str.split('^').str[0]
df_syn['Country of Departure']=df_syn["Aggregated"].str.split('^').str[1]
df_syn['Country of Origin']=df_syn["Aggregated"].str.split('^').str[2]
df_syn['Tax Rate']=df_syn["Aggregated"].str.split('^').str[3]
df_syn['Tax Type']=df_syn["Aggregated"].str.split('^').str[4]
df_syn['Net Mass']=df_syn["Aggregated"].str.split('^').str[5]
df_syn['Fraud']=df_syn["Aggregated"].str.split('^').str[6]
df_syn['Critical Fraud']=df_syn["Aggregated"].str.split('^').str[7]
df_syn = df_syn.drop(['Aggregated'],axis=1)

In [27]:
df_syn

,Declaration ID,Date,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,...,Courier ID,Country of Origin Indicator,HS10 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Net Mass,Fraud,Critical Fraud
0,48649605,2020-10-04,31,B,11,21,19,40,E3Q8ABV,FAQBBWS,...,NaN,G,9031809099,PH,JP,1.3,C,11.9,1,1
1,83990938,2021-05-22,41,B,11,21,44,10,86E3KB0,B6SDM52,...,NaN,E,6106201000,VN,VN,0.0,FVN1,79.8,0,0
2,50667114,2020-09-03,40,B,11,21,43,10,X09KN33,IKSA5YW,...,NaN,E,2906199000,CN,CN,0.0,FCN1,100.0,0,0
3,11390124,2020-08-12,21,B,84,21,11,10,MRTAD2U,0T9AY7N,...,NaN,G,8301401010,CN,CN,8.0,A,6.0,0,0
4,51191871,2020-09-10,20,B,11,21,11,10,IBG3A46,3X6GY58,...,NaN,G,6403999000,IT,IT,0.0,FEU1,2.7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,92685568,2021-02-01,32,B,10,21,45,11,OZB7KED,Y7K2P0D,...,NaN,G,6403999000,MT,CN,13.0,A,100.0,1,1
2996,24146768,2020-10-06,40,B,85,12,44,10,AZI5L7O,89P0ZBF,...,PIAN6F,Y,6206401000,CN,CN,13.0,A,217.0,0,0
2997,98823569,2021-02-01,41,B,10,21,45,41,ACL5K0W,51RX2UZ,...,NaN,G,6204599000,ES,CN,13.0,A,13.2,0,0
2998,41805945,2020-07-30,40,B,11,21,11,11,H1ZUISK,5W9THF3,...,NaN,E,6104520000,FR,TR,13.0,A,10.0,0,0


# Save Generated Data

In [29]:
df_syn.to_csv('./data/df_syn_example.csv', index=False)